In [55]:
import numpy as np
import open3d as o3d

from scipy.spatial.distance import cdist

In [56]:
def draw(*shapes):
    o3d.visualization.draw_geometries(shapes)

In [57]:
mesh = o3d.io.read_triangle_mesh('data/dataset/YellowToy01/yellow_push_toy_1_70000.obj')
point_cloud = o3d.io.read_point_cloud('data/pointcloud_sampled/YellowToy01/inputs/non_deformed_2_correspondences_zoom_2048.ply')
draw(mesh, point_cloud)

In [58]:
mesh_points = np.asarray(mesh.vertices)
cloud_points = np.asarray(point_cloud.points)

# https://stackoverflow.com/questions/8317022/get-intersecting-rows-across-two-2d-numpy-arrays

nrows, ncols = cloud_points.shape
dtype={'names':['f{}'.format(i) for i in range(ncols)],
       'formats':ncols * [cloud_points.dtype]}

common_points = np.intersect1d(cloud_points.view(dtype), mesh_points.view(dtype))

common_points = common_points.view(cloud_points.dtype).reshape(-1, ncols)
assert common_points.shape == cloud_points.shape

In [59]:
mesh_v_merge_map = np.zeros(len(mesh_points), dtype=int)

distances = cdist(mesh_points, cloud_points)
mesh_v_merge_map = distances.argmin(axis=1)
assert len(np.unique(mesh_v_merge_map, axis=0)) == len(cloud_points)

In [61]:
mesh_points = cloud_points[mesh_v_merge_map]
mesh.vertices = o3d.utility.Vector3dVector(mesh_points)
draw(mesh, point_cloud)